In [1]:
# import gym
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt


In [2]:
num_states = 9
num_actions = 4

upper_bound = 1
lower_bound = 0

print("Max Value of Action ->  {}".format(upper_bound))
print("Min Value of Action ->  {}".format(lower_bound))

Max Value of Action ->  1
Min Value of Action ->  0


In [3]:
class OUActionNoise:
    def __init__(self, mean, std_deviation, theta=0.15, dt=1e-2, x_initial=None):
        self.theta = theta
        self.mean = mean
        self.std_dev = std_deviation
        self.dt = dt
        self.x_initial = x_initial
        self.reset()

    def __call__(self):
        # Formula taken from https://www.wikipedia.org/wiki/Ornstein-Uhlenbeck_process.
        x = (
            self.x_prev
            + self.theta * (self.mean - self.x_prev) * self.dt
            + self.std_dev * np.sqrt(self.dt) * np.random.normal(size=self.mean.shape)
        )
        # Store x into x_prev
        # Makes next noise dependent on current one
        self.x_prev = x
        return x

    def reset(self):
        if self.x_initial is not None:
            self.x_prev = self.x_initial
        else:
            self.x_prev = np.zeros_like(self.mean)


In [4]:

class Buffer:
    def __init__(self, buffer_capacity=100000, batch_size=64):

        # Number of "experiences" to store at max
        self.buffer_capacity = buffer_capacity
        # Num of tuples to train on.
        self.batch_size = batch_size

        # Its tells us num of times record() was called.
        self.buffer_counter = 0

        # Instead of list of tuples as the exp.replay concept go
        # We use different np.arrays for each tuple element
        self.state_buffer = np.zeros((self.buffer_capacity, num_states))
        self.action_buffer = np.zeros((self.buffer_capacity, num_actions))
        self.reward_buffer = np.zeros((self.buffer_capacity, 1))
        self.next_state_buffer = np.zeros((self.buffer_capacity, num_states))

    # Takes (s,a,r,s') obervation tuple as input
    def record(self, obs_tuple):
        # Set index to zero if buffer_capacity is exceeded,
        # replacing old records
        index = self.buffer_counter % self.buffer_capacity

        self.state_buffer[index] = obs_tuple[0]
        self.action_buffer[index] = obs_tuple[1]
        self.reward_buffer[index] = obs_tuple[2]
        self.next_state_buffer[index] = obs_tuple[3]

        self.buffer_counter += 1

    # We compute the loss and update parameters
    def learn(self):
        # Get sampling range
        record_range = min(self.buffer_counter, self.buffer_capacity)
        # Randomly sample indices
        batch_indices = np.random.choice(record_range, self.batch_size)

        # Convert to tensors
        state_batch = tf.convert_to_tensor(self.state_buffer[batch_indices])
        action_batch = tf.convert_to_tensor(self.action_buffer[batch_indices])
        reward_batch = tf.convert_to_tensor(self.reward_buffer[batch_indices])
        reward_batch = tf.cast(reward_batch, dtype=tf.float32)
        next_state_batch = tf.convert_to_tensor(self.next_state_buffer[batch_indices])

        # Training and updating Actor & Critic networks.
        # See Pseudo Code.
        with tf.GradientTape() as tape:
            target_actions = target_actor(next_state_batch)
            y = reward_batch + gamma * target_critic([next_state_batch, target_actions])
            critic_value = critic_model([state_batch, action_batch])
            critic_loss = tf.math.reduce_mean(tf.math.square(y - critic_value))

        critic_grad = tape.gradient(critic_loss, critic_model.trainable_variables)
        critic_optimizer.apply_gradients(
            zip(critic_grad, critic_model.trainable_variables)
        )

        with tf.GradientTape() as tape:
            actions = actor_model(state_batch)
            critic_value = critic_model([state_batch, actions])
            # Used `-value` as we want to maximize the value given
            # by the critic for our actions
            actor_loss = -tf.math.reduce_mean(critic_value)

        actor_grad = tape.gradient(actor_loss, actor_model.trainable_variables)
        actor_optimizer.apply_gradients(
            zip(actor_grad, actor_model.trainable_variables)
        )


# This update target parameters slowly
# Based on rate `tau`, which is much less than one.
def update_target(tau):
    new_weights = []
    target_variables = target_critic.weights
    for i, variable in enumerate(critic_model.weights):
        new_weights.append(variable * tau + target_variables[i] * (1 - tau))

    target_critic.set_weights(new_weights)

    new_weights = []
    target_variables = target_actor.weights
    for i, variable in enumerate(actor_model.weights):
        new_weights.append(variable * tau + target_variables[i] * (1 - tau))

    target_actor.set_weights(new_weights)



In [13]:

def get_actor():
    # Initialize weights between -3e-3 and 3-e3
    last_init = tf.random_uniform_initializer(minval=-0.003, maxval=0.003)

    inputs = layers.Input(shape=(num_states,))
    out = layers.Dense(6, activation="relu")(inputs)
    out = layers.BatchNormalization()(out)
    out = layers.Dense(4, activation="relu")(out)
    out = layers.BatchNormalization()(out)
    outputs = layers.Dense(4, activation="tanh", kernel_initializer=last_init)(out)

    # Our upper bound is 2.0 for Pendulum.
    outputs = outputs * upper_bound
    model = tf.keras.Model(inputs, outputs)
    return model


def get_critic():
    # State as input
    state_input = layers.Input(shape=(num_states))
    state_out = layers.Dense(6, activation="relu")(state_input)
    state_out = layers.BatchNormalization()(state_out)
    state_out = layers.Dense(4, activation="relu")(state_out)
    state_out = layers.BatchNormalization()(state_out)

    # Action as input
    action_input = layers.Input(shape=(num_actions))
    action_out = layers.Dense(4, activation="relu")(action_input)
    action_out = layers.BatchNormalization()(action_out)

    # Both are passed through seperate layer before concatenating
    concat = layers.Concatenate()([state_out, action_out])

    out = layers.Dense(4, activation="relu")(concat)
    out = layers.BatchNormalization()(out)
    out = layers.Dense(4, activation="relu")(out)
    out = layers.BatchNormalization()(out)
    outputs = layers.Dense(1)(out)

    # Outputs single value for give state-action
    model = tf.keras.Model([state_input, action_input], outputs)

    return model



In [14]:

def policy(state, noise_object):
    sampled_actions = tf.squeeze(actor_model(state))
    noise = noise_object()
    # Adding noise to action
    sampled_actions = sampled_actions.numpy() + noise

    # We make sure action is within bounds
    legal_action = np.clip(sampled_actions, lower_bound, upper_bound)

    return [np.squeeze(legal_action)]



In [18]:
std_dev = 0.1
ou_noise = OUActionNoise(mean=np.zeros(1), std_deviation=float(std_dev) * np.ones(1))

actor_model = get_actor()
critic_model = get_critic()

target_actor = get_actor()
target_critic = get_critic()

# Making the weights equal initially
target_actor.set_weights(actor_model.get_weights())
target_critic.set_weights(critic_model.get_weights())

# Learning rate for actor-critic models
critic_lr = 0.0005
actor_lr = 0.0003

critic_optimizer = tf.keras.optimizers.Adam(critic_lr)
actor_optimizer = tf.keras.optimizers.Adam(actor_lr)

total_episodes = 1000
# Discount factor for future rewards
gamma = 0.99
# Used to update target networks
tau = 0.005

buffer = Buffer(50000, 32)


In [19]:
#helper functions using simulator
import simulator
import os
import json
import math
import numpy as np
import random
from shutil import copyfile
import matplotlib
import matplotlib.pyplot as plt

import pickle

lambda_h = 1.0
lambda_i = 0.5
lambda_q = 0.3
lambda_c = 0.2

def compute_score(I, Q):
	theta_i = 1000.0
	theta_q = 100000.0
	q_w = 1.0

	return math.exp(I*1.0/theta_i) + q_w * math.exp(Q*1.0/theta_q)

def prob_infected_from_acq(user_id, pc=0.0015):

	residental_acq = engine.get_individual_residential_acq(user_id)
	working_acq = engine.get_individual_working_acq(user_id)

	area_ids = engine.get_individual_visited_history(user_id)
	count_same_places = [0 for _ in area_ids]
	for acq_id in residental_acq:
		acq_area_ids = engine.get_individual_visited_history(acq_id)
		for idx, (area_id, acq_area_id) in enumerate(zip(area_ids, acq_area_ids)):
			if area_id == acq_area_id and area_id > -1:
				count_same_places[idx] +=1

	for acq_id in working_acq:
		acq_area_ids = engine.get_individual_visited_history(acq_id)
		for idx, (area_id, acq_area_id) in enumerate(zip(area_ids, acq_area_ids)):
			if area_id == acq_area_id and area_id > -1: 
				count_same_places[idx] +=1

	p = 1 #prob of not being transmitted
	for c in count_same_places:
		if c > 0:
			p = p*((1-pc)**c)
	return 1-p

def interven(user_id, engine, isolation_threshold, quarantine_threshold, confine_threshold, confine_f2_threshold,
	isolation_day=1, quarantine_day=1, confine_day=1, confine_f2_day=1):
	infection_state = engine.get_individual_infection_state(user_id)
	if infection_state  == 3 or infection_state == 4:

		residental_acq = engine.get_individual_residential_acq(user_id)
		working_acq = engine.get_individual_working_acq(user_id)

		#act the acq
		for acq_id in residental_acq:
			if engine.get_individual_intervention_state(acq_id) == 1:
				prob_infected = prob_infected_from_acq(acq_id)
				# print("Prob of being infected = ", prob_infected)
				# print("*******************************************+++++++++++++++++++++")
				if prob_infected > isolation_threshold:
					engine.set_individual_isolate_days({acq_id: isolation_day})
					
				elif prob_infected > quarantine_threshold:
					engine.set_individual_quarantine_days({acq_id: quarantine_day})
					
				elif prob_infected > confine_threshold:
					engine.set_individual_confine_days({acq_id: confine_day})
					
				#quarantine f2
				if prob_infected > confine_f2_threshold:
					residental_acq_f2 = engine.get_individual_residential_acq(acq_id)
					working_acq_f2 = engine.get_individual_working_acq(acq_id)

					
					for f2 in residental_acq_f2:
						if engine.get_individual_intervention_state(f2) == 1:
							# engine.set_individual_quarantine_days({f2: quarantine_day})
							engine.set_individual_confine_days({f2: confine_f2_day})
							
					for f2 in working_acq_f2:
						if engine.get_individual_intervention_state(f2) == 1:
							# engine.set_individual_quarantine_days({f2: quarantine_day})
							engine.set_individual_confine_days({f2: confine_f2_day})
								

		for acq_id in working_acq:
			if engine.get_individual_intervention_state(acq_id) == 1:
				prob_infected = prob_infected_from_acq(acq_id)
				# print("Prob of being infected = ", prob_infected)
				# print("*******************************************+++++++++++++++++++++")
				if prob_infected > isolation_threshold:
					engine.set_individual_isolate_days({acq_id: isolation_day})
					
				elif prob_infected > quarantine_threshold:
					engine.set_individual_quarantine_days({acq_id: quarantine_day})
					
				elif prob_infected > confine_threshold:
					engine.set_individual_confine_days({acq_id: confine_day})
					
				if prob_infected > confine_f2_threshold:
					#quarantine f2
					residental_acq_f2 = engine.get_individual_residential_acq(acq_id)
					working_acq_f2 = engine.get_individual_working_acq(acq_id)

					
					for f2 in residental_acq_f2:
						if engine.get_individual_intervention_state(f2) == 1:
							# engine.set_individual_quarantine_days({f2: quarantine_day})
							engine.set_individual_confine_days({f2: confine_f2_day})
							
					for f2 in working_acq_f2:
						if engine.get_individual_intervention_state(f2) == 1:
							# engine.set_individual_quarantine_days({f2: quarantine_day})
							engine.set_individual_confine_days({f2: confine_f2_day})
								


	if (infection_state == 3 or infection_state == 4):
		if engine.get_individual_intervention_state(user_id) < 5:
			#hospitalize user
			
			engine.set_individual_to_treat({user_id: True})

            
def get_state(engine):
    cur_day = engine.get_current_day()*1.0/60.0 #normalize
    total_case = 0
    new_case = 0
    f1 = set()
    f2 = set()
    for user_id in range(10000):
        infection_state = engine.get_individual_infection_state(user_id)
        if infection_state  == 3 or infection_state == 4:
            total_case += 1
            if engine.get_individual_intervention_state(user_id) < 5:
                new_case +=1
                
            residental_acq = engine.get_individual_residential_acq(user_id)
            working_acq = engine.get_individual_working_acq(user_id)
            f1.update(residental_acq)
            f1.update(working_acq)
            
            for acq in residental_acq:
                f2.update([x for x in engine.get_individual_residential_acq(acq) if x not in f1])
                
                f2.update([x for x in engine.get_individual_working_acq(acq) if x not in f1])
            for acq in working_acq:
                f2.update([x for x in engine.get_individual_residential_acq(acq) if x not in f1])
                f2.update([x for x in engine.get_individual_working_acq(acq) if x not in f1])
    all_prob_f1 = [prob_infected_from_acq(x) for x in f1]
    if len(all_prob_f1) == 0:
        all_prob_f1 = [0]
            
    return [cur_day, total_case*1.0/10000, new_case*1.0/10000, len(f1)*1.0/10000, len(f2)*1.0/10000, 
           np.mean(all_prob_f1), np.min(all_prob_f1), np.max(all_prob_f1), np.median(all_prob_f1)]

            
                

    

In [ ]:
# To store reward history of each episode
ep_reward_list = []
# To store average reward history of last few episodes
avg_reward_list = []

# Takes about ? min to train
for ep in range(total_episodes):
    
    #create simulator
    I , Q = 0,0 #for computing score
    engine = simulator.Engine(thread_num=1, write_mode="append", specified_run_name="test")
    engine.reset()
    score = compute_score(I, Q)
    
    #compute state 
    prev_state = get_state(engine)
    episodic_reward = 0
    period = 840
    infected_set = set()
    for i in range(period):
        # Uncomment this to see the Actor in action
        # But not in a python notebook.
        # env.render()
        if i % 14 == 0:
            tf_prev_state = tf.expand_dims(tf.convert_to_tensor(prev_state), 0)

            action = policy(tf_prev_state, ou_noise)[0]
#             print("action = ", action)
            
            #apply action to environment
            [isolation_threshold, quarantine_threshold, confine_threshold, confine_f2_threshold] = action
#             print("isolation threshold = ", isolation_threshold)
            for user_id in range(10000):
                infection_state = engine.get_individual_infection_state(user_id)
                if infection_state  == 3 or infection_state == 4:
                    infected_set.add(user_id)
                interven(user_id,engine, isolation_threshold, quarantine_threshold, 
                         confine_threshold, confine_f2_threshold)
            I = len(infected_set)
            Q += lambda_h * engine.get_hospitalize_count()
            Q += lambda_i * engine.get_isolate_count()
            Q += lambda_q * engine.get_quarantine_count()
            Q += lambda_c * engine.get_confine_count()
            new_score = compute_score(I, Q)
            reward = score - new_score
            score = new_score
            state = get_state(engine)
        
            # Recieve state and reward from environment.
#             state, reward, done, info = env.step(action)

            buffer.record((prev_state, action, reward, state))
            episodic_reward += reward

            buffer.learn()
            update_target(tau)


            prev_state = state
        #next time step
        engine.next_step()
        engine.get_current_time()

    ep_reward_list.append(episodic_reward)

    # Mean of last 40 episodes
    avg_reward = np.mean(ep_reward_list[-40:])
    print("Episode * {} * Avg Reward is ==> {}".format(ep, avg_reward))
    avg_reward_list.append(avg_reward)

# Plotting graph
# Episodes versus Avg. Rewards
plt.plot(avg_reward_list)
plt.xlabel("Episode")
plt.ylabel("Avg. Epsiodic Reward")
plt.show()


Episode * 0 * Avg Reward is ==> -0.3670430505541322
Episode * 1 * Avg Reward is ==> -0.34874998908356813
Episode * 2 * Avg Reward is ==> -0.37553546641960295
Episode * 3 * Avg Reward is ==> -0.4149709573114735
Episode * 4 * Avg Reward is ==> -0.45507946448239006
Episode * 5 * Avg Reward is ==> -0.44498652410611056
Episode * 6 * Avg Reward is ==> -0.4707889190561095
Episode * 7 * Avg Reward is ==> -0.47297828114151025
Episode * 8 * Avg Reward is ==> -0.4730149771399233
Episode * 9 * Avg Reward is ==> -0.46982660990810887
Episode * 10 * Avg Reward is ==> -0.4818130181027964
Episode * 11 * Avg Reward is ==> -0.48536132195742643


In [ ]:
# Save the weights
actor_model.save_weights("actor.h5")
critic_model.save_weights("critic.h5")

target_actor.save_weights("target_actor.h5")
target_critic.save_weights("target_critic.h5")